In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

respondent IDs for each lender

In [2]:
# banks
USBank= 504713  # US Bank
Wells=451965  #Wells Fargo
Bell=19581    # Bell Bank

# online lenders
Quicken=7197000003        #Quicken Loan
AMEC=411941324             #American Mortgage & Equity Consultants Inc.
Guaranteed=364327855      # Guaranteed Rate


In [3]:
def LogRegress(X_train, y_train,X_test,y_test):
#####################################
# use Logistic Regression to train and test a model
# returning test score

    # create model
    LRmodel = LogisticRegression()
    # fit model with training data
#     print("X-train",X_train.head())
#     print("y_train",y_train.head())
    LRmodel.fit(X_train, y_train)
    # validate the model using testing data
    test_score=LRmodel.score(X_test, y_test)
    params=LRmodel.get_params(deep=False)
#    print("test_score",test_score)
#    print("params=",params)
    return test_score

In [4]:
def AssignData(set, data_df):
##############################################
# choose the lender group for test
# set = 0 use all of data_df this is all lenders
# set = 1 select from data_df for 'respondent id' in Banks
# set = 2 select from data_df for 'respondent id' in Online
# set > 3 select from data_df for 'respondent id' = set
# split for train and test data
# call LogRegress

    #reduce the data used for set > 0 to a subset of lenders or a particular lender
    if set > 3 :
        data_df=data_df.loc[data_df['respondent id'] == set,:]
    elif set == 2:
        data_df=data_df.loc[(data_df['respondent id'] == Quicken) | (data_df['respondent id'] == AMEC) | (data_df['respondent id'] == Guaranteed), :]
    elif set == 1:
        data_df=data_df.loc[(data_df['respondent id'] == USBank) | (data_df['respondent id'] == Wells) | (data_df['respondent id'] == Bell), :]
           
        # Assign X (data) and y (target)
    X=data_df.drop(["action modified","respondent id"],axis=1)
    y=data_df["action modified"]
#    print("shape", X.shape, y.shape)

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

    return LogRegress(X_train, y_train,X_test,y_test)

In [5]:
def AssignData2(set, data_df,test_df):
##############################################
# FOR USE WHEN TRAINING AND TESTING ON DIFFERENT SETS
# choose the lender group for BOTH train and test data
# set = 0 use all of data_df this is all lenders
# set = 1 select from data_df for 'respondent id' in Banks
# set = 2 select from data_df for 'respondent id' in Online
# set > 3 select from data_df for 'respondent id' = set
# call LogRegress


    #reduce the data used for set > 0 to a subset of lenders or a particular lender
    if set > 3 :
        data_df=data_df.loc[data_df['respondent id'] == set,:]
        test_df=test_df.loc[test_df['respondent id'] == set,:]
    elif set == 2:
        data_df=data_df.loc[(data_df['respondent id'] == Quicken) | (data_df['respondent id'] == AMEC) | (data_df['respondent id'] == Guaranteed), :]
        test_df=test_df.loc[(test_df['respondent id'] == Quicken) | (test_df['respondent id'] == AMEC) | (test_df['respondent id'] == Guaranteed), :]
    elif set == 1:
        data_df=data_df.loc[(data_df['respondent id'] == USBank) | (data_df['respondent id'] == Wells) | (data_df['respondent id'] == Bell), :]
        test_df=test_df.loc[(test_df['respondent id'] == USBank) | (test_df['respondent id'] == Wells) | (test_df['respondent id'] == Bell), :]
           
        # Assign X (data) and y (target) for training
    X_train=data_df.drop(["action modified","respondent id"],axis=1)
    y_train=data_df["action modified"]
    
            # Assign X (data) and y (target) for test
    X_test=test_df.drop(["action modified","respondent id"],axis=1)
    y_test=test_df["action modified"]
#    print("shape", X.shape, y.shape)


    return LogRegress(X_train, y_train,X_test,y_test)

In [6]:
def LenderTest(test_data):
#################################################
# goes through the 9 tests of lender groups with one version of the test data
# 0- All data
# 1- all banks
# 2- all online lenders
# 3 USBANK 4- Wells 5- Bell 
# 6- Quicken 7- AMEC 8- Guaranteed
# returns the results in list
############################################################
    result_list=[]
    # get result for all data
    result_list.append(AssignData(0,test_data))
    # get result for all banks
    result_list.append(AssignData(1,test_data))
    #result for all online
    result_list.append(AssignData(2,test_data))
    # result for USBank, Wells, Bell 
    result_list.append(AssignData(USBank,test_data))
    result_list.append(AssignData(Wells,test_data))
    result_list.append(AssignData(Bell,test_data))
    #result for Quicken, AMEC, Guaranteed
    result_list.append(AssignData(Quicken,test_data))
    result_list.append(AssignData(AMEC,test_data))
    result_list.append(AssignData(Guaranteed,test_data))
    return result_list

In [7]:
def LenderTest2(train_data,test_data):
#################################################
# FOR USE WHEN TRAINING AND TESTING ON DIFFERENT SETS
# goes through the 9 tests  with one version of the test data
# 0- All data
# 1- all banks
# 2- all online lenders
# 3 USBANK 4- Wells 5- Bell 
# 6- Quicken 7- AMEC 8- Guaranteed
# returns the results in list
############################################################
    result_list=[]
    # get result for all data
    result_list.append(AssignData2(0,train_data,test_data))
    # get result for all banks
    result_list.append(AssignData2(1,train_data,test_data))
    #result for all online
    result_list.append(AssignData2(2,train_data,test_data))
    # result for USBank, Wells, Bell 
    result_list.append(AssignData2(USBank,train_data,test_data))
    result_list.append(AssignData2(Wells,train_data,test_data))
    result_list.append(AssignData2(Bell,train_data,test_data))
    #result for Quicken, AMEC, Guaranteed
    result_list.append(AssignData2(Quicken,train_data,test_data))
    result_list.append(AssignData2(AMEC,train_data,test_data))
    result_list.append(AssignData2(Guaranteed,train_data,test_data))
    return result_list

# main body of code 
read in data file

prepare training data sets

In [9]:
file="AllData2017.csv"
loan_data=pd.read_csv(file)

loan_data=loan_data[["respondent id","agency code","property type","loan purpose","loan amount","county",\
                     "applicant race 1","sex","type of purchaser", "hoepa status","lien status","loan type modified",\
                     "action modified","income cleaned","income loan ratio"]]

sexless_data=loan_data.drop(["sex"], axis=1)
raceless_data=loan_data.drop(["applicant race 1"], axis=1)
loantypeless_data=loan_data.drop(["loan type modified"], axis=1) 
incomeratioless_data=loan_data.drop(["income loan ratio"], axis=1) 


prepare unique group test data sets 

In [10]:
# single gender  - genderless data
female_data=loan_data[loan_data["sex"] == 2]
female_data=female_data.drop(["sex"],axis=1)
male_data=loan_data[loan_data["sex"] == 1]
male_data=male_data.drop(["sex"],axis=1)

#single race - raceless data
indian_data=loan_data[loan_data["applicant race 1"] == 1]
indian_data=indian_data.drop(["applicant race 1"],axis=1)
asian_data=loan_data[loan_data["applicant race 1"] == 2]
asian_data=asian_data.drop(["applicant race 1"],axis=1)
black_data=loan_data[loan_data["applicant race 1"] == 3]
black_data=black_data.drop(["applicant race 1"],axis=1)
hawaii_data=loan_data[loan_data["applicant race 1"] == 4]
hawaii_data=hawaii_data.drop(["applicant race 1"],axis=1)
white_data=loan_data[loan_data["applicant race 1"] == 5]
white_data=white_data.drop(["applicant race 1"],axis=1)

# single loan type - loantypeless data
govtbacked_data=loan_data[loan_data["loan type modified"] == 0]
govtbacked_data=govtbacked_data.drop(["loan type modified"],axis=1)
conventional_data=loan_data[loan_data["loan type modified"] == 1]
conventional_data=conventional_data.drop(["loan type modified"],axis=1)

# single income ratio group - income ratio less data  (income ratio = (income / loan amount) * 100 )
lowincomeratio_data=loan_data[loan_data["income loan ratio"] <= 100]
lowincomeratio_data=lowincomeratio_data.drop(["income loan ratio"],axis=1)
medincomeratio_data=loan_data[(loan_data["income loan ratio"] > 100) & (loan_data["income loan ratio"] < 200) ]
medincomeratio_data=medincomeratio_data.drop(["income loan ratio"],axis=1)
highincomeratio_data=loan_data[loan_data["income loan ratio"] >= 200]
highincomeratio_data=highincomeratio_data.drop(["income loan ratio"],axis=1)

# run tests and append lender sets to dataframe
for each training group run with same train and test data

next run with training data and unique group test data

In [11]:
result_df=pd.DataFrame(index=["all","banks","online","usbank","wells","bell","quicken","amec","guaranteed"])

result_df["all_fields"]=LenderTest(loan_data)

#results for the effectof gender 
result_df["sexless"]=LenderTest(sexless_data)
result_df["female"]=LenderTest2(sexless_data,female_data)
result_df["male"]=LenderTest2(sexless_data,male_data)

#results for the effect of race
result_df["raceless"]=LenderTest(raceless_data)
result_df["indian"]=LenderTest2(raceless_data,indian_data)
result_df["asian"]=LenderTest2(raceless_data,asian_data)
result_df["black"]=LenderTest2(raceless_data,black_data)
result_df["hawaiian"]=LenderTest2(raceless_data,hawaii_data)
result_df["white"]=LenderTest2(raceless_data,white_data)

#results for the effect of loan type
result_df["loantypeless"]=LenderTest(loantypeless_data)
result_df["govtbacked"]=LenderTest2(loantypeless_data,govtbacked_data)
result_df["conventional"]=LenderTest2(loantypeless_data,conventional_data)

# results for the effect of income / loan ratio 
result_df["incomeratioless"]=LenderTest(incomeratioless_data)
result_df["low income ratio"]=LenderTest2(incomeratioless_data,lowincomeratio_data)
result_df["med income ratio"]=LenderTest2(incomeratioless_data,medincomeratio_data)
result_df["high income ratio"]=LenderTest2(incomeratioless_data,highincomeratio_data)
result_df

,all_fields,sexless,female,male,raceless,indian,asian,black,hawaiian,white,loantypeless,govtbacked,conventional,incomeratioless,low income ratio,med income ratio,high income ratio
all,0.879997,0.878831,0.878677,0.889225,0.880247,0.800813,0.861798,0.851906,0.851852,0.889715,0.870919,0.928872,0.864688,0.878081,0.751616,0.864714,0.917622
banks,0.860973,0.860852,0.857361,0.874274,0.857819,0.740331,0.845657,0.792291,0.782609,0.872076,0.855514,0.915912,0.854105,0.855392,0.742607,0.849616,0.898414
online,0.962815,0.962815,0.969295,0.965467,0.962815,0.953846,0.968992,0.976744,0.948718,0.972316,0.962815,0.965079,0.960772,0.962815,0.929955,0.954706,0.966916
usbank,0.786088,0.787958,0.773760,0.804874,0.786088,0.655172,0.769892,0.654362,0.593750,0.803579,0.777861,0.823925,0.776181,0.773373,0.698541,0.796510,0.794775
wells,0.886903,0.887170,0.872621,0.898369,0.884503,0.774194,0.882629,0.812500,0.833333,0.893094,0.886636,0.921663,0.887552,0.888504,0.777082,0.920269,0.915732
bell,0.976387,0.976387,0.977760,0.975346,0.977485,0.900000,0.979239,0.965116,0.714286,0.977546,0.973641,0.964082,0.976479,0.974739,0.967480,0.973464,0.974613
quicken,0.932252,0.932252,0.934077,0.928571,0.932252,0.923077,0.908397,0.935714,0.866667,0.938582,0.932252,0.938954,0.929838,0.932252,0.905350,0.935346,0.934488
amec,0.999253,0.999253,0.999463,1.000000,0.999253,1.000000,1.000000,0.995745,1.000000,1.000000,0.999253,0.999480,0.999708,0.999253,1.000000,1.000000,0.999544
guaranteed,0.996979,0.996979,0.997845,0.995204,0.996979,1.000000,1.000000,1.000000,1.000000,0.996488,0.996979,0.994751,0.996819,0.996979,0.987500,1.000000,0.996020
